In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import scipy
import time
from PIL import Image,ImageOps
from keras.datasets import mnist

from keras import layers
import keras
from keras.layers import Dropout
from matplotlib import pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import initializers
from keras import optimizers
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.layers.normalization import BatchNormalization
from array import array
import cv2
import os
import random


In [ ]:

(train_imagesall, train_labels), (test_imagesall, test_labels) = mnist.load_data()

np.random.shuffle(train_labels)


train_imagesall=train_imagesall[:,:,:,np.newaxis]
test_imagesall=test_imagesall[:,:,:,np.newaxis]
train_images=([])
test_images1=([])
for i in range(0,60000):
  train_images.append(np.concatenate((train_imagesall[i],train_imagesall[i],train_imagesall[i]),axis=2))
for i in range(0,10000):
  test_images1.append(np.concatenate((test_imagesall[i],test_imagesall[i],test_imagesall[i]),axis=2))

train_images=np.array(train_images)
test_images1=np.array(test_images1)


val_images = test_images1[0:2000,:,:]
val_labels = test_labels[0:2000]
test_images = test_images1[2000:,:,:]
test_labels = test_labels[2000:]

train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5
val_images = (val_images / 255) - 0.5

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)


model = keras.Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(28,28,3)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=10, activation = 'softmax'))

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

model.compile(
  optimizer=adam,
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# obtain the summary
#saved_model.summary()


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
mc = ModelCheckpoint('/content/gdrive/MyDrive/models/task3_random.h5', monitor='val_accuracy', mode='max', 
                     verbose=1, save_best_only=True)

# Train the model.

start_time = time.time()
history=model.fit(
  train_images,
  to_categorical(train_labels),
  validation_data=(val_images, to_categorical(val_labels)),  
  epochs=5,
  batch_size=32,
  shuffle = True,
  callbacks=[es,mc],
)


scores = model.evaluate(
  test_images,
  to_categorical(test_labels)
)

print('Test accuracy:', scores[1])
print("--- %s seconds ---" % (time.time() - start_time))



es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
mc = ModelCheckpoint(r'/content/gdrive/MyDrive/models/task3_pretrained.h5', monitor='val_accuracy', mode='max', 
                     verbose=1, save_best_only=True)
saved_model=load_model("/content/gdrive/MyDrive/models/pretrained_2.h5")
start_time = time.time()

history=saved_model.fit(
  train_images,
  to_categorical(train_labels),
  validation_data=(val_images, to_categorical(val_labels)),  
  epochs=5,
  batch_size=32,
  shuffle = True,
  callbacks=[es,mc],
)


scores = saved_model.evaluate(
  test_images,
  to_categorical(test_labels)
)

print('Test accuracy:', scores[1])
print("--- %s seconds ---" % (time.time() - start_time))


Epoch 1/5
1875/1875 [==============================] - 27s 14ms/step - loss: 2.3023 - accuracy: 0.1078 - val_loss: 2.3009 - val_accuracy: 0.1170

Epoch 00001: val_accuracy improved from -inf to 0.11700, saving model to /content/gdrive/MyDrive/models/task3_random.h5
Epoch 2/5
1875/1875 [==============================] - 26s 14ms/step - loss: 2.3014 - accuracy: 0.1141 - val_loss: 2.3013 - val_accuracy: 0.1170

Epoch 00002: val_accuracy did not improve from 0.11700
Epoch 3/5
1875/1875 [==============================] - 26s 14ms/step - loss: 2.3015 - accuracy: 0.1100 - val_loss: 2.3006 - val_accuracy: 0.1170

Epoch 00003: val_accuracy did not improve from 0.11700
Epoch 4/5
1875/1875 [==============================] - 26s 14ms/step - loss: 2.3015 - accuracy: 0.1121 - val_loss: 2.3032 - val_accuracy: 0.1170

Epoch 00004: val_accuracy did not improve from 0.11700
Epoch 5/5
1875/1875 [==============================] - 26s 14ms/step - loss: 2.3010 - accuracy: 0.1106 - val_loss: 2.3030 - val_acc